# Ripple perturbation of a graphene sheet.

## 0. Introduction

This notebook demonstrates how to recreate a material from the following manuscript:

> Thompson-Flagg, R. C., Moura, M. J. B., & Marder, M.
> Rippling of graphene. 
> EPL (Europhysics Letters), 85(4), 46002. 2009
> [DOI: 10.1209/0295-5075/85/46002](https://doi.org/10.1209/0295-5075/85/46002)


Recreating material from Fig. 1:

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/perturbation_ripple_graphene/0-figure-from-manuscript.webp" alt="Rippling of graphene" width="400"/>


## 1. Prepare the Environment

### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples")

### 1.2. Set Nanoribbon and Perturbation Parameters

In [ ]:
# Set whether to preserve geodesic distance and scale the cell accordingly to match PBC
PRESERVE_GEODESIC_DISTANCE = False

NANORIBBON_SIZE = 40 # in unit cells, lateral width and length of the nanoribbon
VACUUM_SIZE = 10 # in unit cells, lateral width and length of the vacuum region

# Set whether to use Cartesian coordinates for the perturbation function
USE_CARTESIAN_COORDINATES = False
MATERIAL_NAME = "Graphene"

### 1.3. Define Custom Perturbation Function
Provide a [SymPy](https://docs.sympy.org/latest/tutorials/intro-tutorial/intro.html) expression for the perturbation function. The expression should be a function of `x`, `y` and `z` variables.

In [ ]:
import sympy as sp

# Variables for the perturbation function (for SymPy)
variable_names = ["x", "y", "z"]
x, y, z = sp.symbols(variable_names)

# Set the parameters for the perturbation function
AMPLITUDE = 0.09  # Ripple amplitude
WAVELENGTH = 0.2  # Wavelength of ripples
EDGE_WIDTH = 0.25  # Width of edge effect
PHASE_X = 0.0  # Phase shift for x direction
PHASE_Y = sp.pi/2  # Phase shift for y direction

# Create edge masks for both x and y using polynomial functions
left_edge_x = sp.Max(0, (EDGE_WIDTH - x) / EDGE_WIDTH)
right_edge_x = sp.Max(0, (x - (1 - EDGE_WIDTH)) / EDGE_WIDTH)
left_edge_y = sp.Max(0, (EDGE_WIDTH - y) / EDGE_WIDTH)
right_edge_y = sp.Max(0, (y - (1 - EDGE_WIDTH)) / EDGE_WIDTH)

# Combine edge masks
edge_mask_x = left_edge_x + right_edge_x
edge_mask_y = left_edge_y + right_edge_y
edge_mask = edge_mask_x + edge_mask_y

# Wave pattern
wave_pattern = (
    sp.sin(2 * sp.pi * x / WAVELENGTH + PHASE_X) * 
    sp.sin(2 * sp.pi * y / WAVELENGTH + PHASE_Y)
)

# Combine waves with edge mask
custom_sympy_function = AMPLITUDE * wave_pattern * edge_mask

### 1.4. Get input materials

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

material = Material(Materials.get_by_name_first_match(MATERIAL_NAME))

### 1.5. Create and preview Nanoribbon

In [ ]:
from mat3ra.made.tools.build.nanoribbon import create_nanoribbon, NanoribbonConfiguration
from utils.visualize import visualize_materials as visualize

config = NanoribbonConfiguration(material=material, width=NANORIBBON_SIZE, length=NANORIBBON_SIZE, vacuum_width=VACUUM_SIZE, vacuum_length=VACUUM_SIZE)

supercell = create_nanoribbon(config)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create a target material
### 2.1. Set custom perturbation parameters


In [ ]:
from mat3ra.made.tools.build.perturbation import CellMatchingDistancePreservingSlabPerturbationBuilder, \
    PerturbationConfiguration, SlabPerturbationBuilder
from mat3ra.made.tools.utils.perturbation import PerturbationFunctionHolder

custom_perturbation_function = PerturbationFunctionHolder(function=custom_sympy_function,
                                                          variables=variable_names)
configuration_custom = PerturbationConfiguration(
    material=supercell,
    perturbation_function_holder=custom_perturbation_function,
    use_cartesian_coordinates=USE_CARTESIAN_COORDINATES)

if PRESERVE_GEODESIC_DISTANCE:
    builder = CellMatchingDistancePreservingSlabPerturbationBuilder()
else:
    builder = SlabPerturbationBuilder()

### 2.2. Apply perturbation to the material

In [ ]:
from mat3ra.made.tools.build.perturbation import create_perturbation

material_with_custom_perturbation = create_perturbation(configuration=configuration_custom, builder=builder)

### 3. Visualize the Material

In [ ]:
visualize([
    {"material": material_with_custom_perturbation, "title": f"Material with custom perturbation"},
    {"material": material_with_custom_perturbation, "title": f"Material with custom perturbation", "rotation": "-90x"}
])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(material_with_custom_perturbation)
download_content_to_file(material_with_custom_perturbation, f"{MATERIAL_NAME}_edge_perturbation.json")